In [1]:
using Revise

using RigidBodyDynamics
using RigidBodyDynamics: Bounds

using Plots
using BilevelTrajOpt

In [2]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)
body = findbody(mechanism, "ball")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100)
μ = 0.5
motion_type = :xyz
point = Point3D(default_frame(findbody(mechanism, "floor")), [0.,0.,0.])
normal = FreeVector3D(default_frame(findbody(mechanism, "floor")), [0.,0.,1.])
hs = HalfSpace(point, normal)
floor = Obstacle(hs, μ, motion_type)
obstacles = [floor]
env = parse_contacts(mechanism, urdf, obstacles)
x0 = MechanismState(mechanism)
Δt = 0.005
sim_data = get_sim_data(x0,env,Δt,true);

In [64]:
q0 = [1., 0., 0., 0., 0., 0., 0.]
v0 = [0., 0., 0., 0.1, 0.1, 0.]

u0 = zeros(sim_data.num_v)
set_configuration!(x0, q0)
set_velocity!(x0, v0)
setdirty!(x0)
ctrl! = (u,t,x) -> u[:] = 0.
traj = BilevelTrajOpt.simulate(x0,env,sim_data.Δt,1,ctrl!,implicit_contact=false)
qnext = traj[1:sim_data.num_q,2]
vnext = traj[sim_data.num_q+1:sim_data.num_q+sim_data.num_v,2]
x_sol_exp = traj[sim_data.num_q+sim_data.num_v+2:end,2]

# qnext = q0
# vnext = v0

τ_ip, x_sol_ip, λ_ip, μ_ip, fx_ip = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=true);

τ_auglag, x_sol_auglag, λ_auglag, μ_auglag, fx_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=false);

display("Explicit")
display(x_sol_exp)
display("IP")
display(x_sol_ip)
display("Aug Lag")
display(x_sol_auglag)

display("torques")
display(τ_ip)
display(τ_auglag)

elapsed time: 0.005019097 seconds
elapsed time: 0.004617305 seconds
elapsed time: 0.004854205 seconds
elapsed time: 0.004526943 seconds
elapsed time: 0.004763152 seconds
elapsed time: 0.004541691 seconds
elapsed time: 0.004544326 seconds
elapsed time: 0.004696385 seconds
elapsed time: 0.004375736 seconds
elapsed time: 0.004627404 seconds
elapsed time: 0.004708718 seconds
elapsed time: 0.004314711 seconds
elapsed time: 0.004390424 seconds
elapsed time: 0.004622485 seconds


elapsed time: 0.00458854 seconds
elapsed time: 0.005053112 seconds
elapsed time: 0.004875835 seconds
elapsed time: 0.004867329 seconds
elapsed time: 0.0050942 seconds
elapsed time: 0.004829287 seconds
elapsed time: 0.004816472 seconds
elapsed time: 0.004800282 seconds
elapsed time: 0.004689321 seconds
Solve_Succeeded
Not_Enough_Degrees_Of_Freedom

"Explicit"

6-element Array{Float64,1}:
 2.34941e-8
 2.34941e-8
 0.25      
 0.25      
 0.0877375 
 1.0       

"IP"

...

"Aug Lag"

6-element Array{Float64,1}:
 7.93053e-9
 7.95988e-9
 0.25      
 0.25      
 0.0877375 
 1.0       

"torques"

6-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

6-element Array{Float64,1}:
  0.0   
  0.0   
  0.0   
  2.4525
  2.4525
 -9.81  

In [58]:
J_auto = ForwardDiff.jacobian(qv -> solve_implicit_contact_τ(sim_data,q0,v0,u0,qv[1:sim_data.num_q],qv[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)[2], vcat(qnext,vnext))

6×13 Array{Float64,2}:
 0.0  -0.0122564     0.0122564   …    0.999501    0.999502   0.499752  
 0.0  -0.0122561     0.0122561        0.999477    0.999477   0.499739  
 0.0  -0.0906845     0.0906845      -12.9921      7.39527    3.69764   
 0.0  -0.0906842     0.0906842        7.39525   -12.9921     3.69763   
 0.0   2.34523e-10  -2.3597e-10       0.5         0.5       -1.45653e-8
 0.0  -0.460786      0.460786    …   -3.19788    -3.19788   18.7884    

In [65]:
τ_auglag, x_sol_auglag, λ_auglag, μ_auglag, fx_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=false);

# autodiff 
J_auto = ForwardDiff.jacobian(qv -> solve_implicit_contact_τ(sim_data,q0,v0,u0,qv[1:sim_data.num_q],qv[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)[2], vcat(qnext,vnext))
# Jt_auto = ForwardDiff.jacobian(qv -> solve_implicit_contact_τ(sim_data,q0,v0,u0,qv[1:sim_data.num_q],qv[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)[1], vcat(qnext,vnext))

# numerical
ϵ = 1e-8
J_num = zeros(size(J_auto))
# Jt_num = zeros(size(J_auto))
# Jt_num_ip = zeros(size(J_auto))
qv = vcat(qnext,vnext)
for i = 1:length(qv)
    δ = zeros(length(qv))
    δ[i] = ϵ 
    τ, x, λ, μ, fx = solve_implicit_contact_τ(sim_data,q0,v0,u0,(qv+δ)[1:sim_data.num_q],(qv+δ)[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)
    J_num[:,i] = (x - x_sol_auglag)/ϵ
#     Jt_num[:,i] = (τ - τ_auglag)/ϵ
#     τ, x, λ, μ, fx = solve_implicit_contact_τ(sim_data,q0,v0,u0,(qv+δ)[1:sim_data.num_q],(qv+δ)[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=true)
#     Jt_num_ip[:,i] = (τ - τ_auglag)/ϵ
end

display(J_auto)
display(J_num)

# display(Jt_auto)
# display(Jt_num)
# display(Jt_num_ip)

6×13 Array{Float64,2}:
 0.0  -0.0135775    0.0135775   …    1.1072      1.1072     0.553618 
 0.0  -0.0135771    0.0135771        1.1072      1.1072     0.553602 
 0.0  -0.100459     0.100459       -12.195       8.19238    4.09618  
 0.0  -0.100459     0.100459         8.19237   -12.195      4.09618  
 0.0   8.72873e-5  -8.72174e-5       0.500858    0.500858  -0.0035544
 0.0  -0.456558     0.456558    …   -3.54258    -3.54258   18.616    

6×13 Array{Float64,2}:
 0.0  -0.0185299     0.00553914    0.00044324   …    1.11388    0.548876   
 0.0  -0.0190978     0.00923254   -0.00305802        1.1071     0.546554   
 0.0  -0.0981233     0.105992     -0.49522           8.19176    4.09663    
 0.0  -0.0945507     0.10802       0.501013        -12.1943     4.1035     
 0.0   0.000206446  -0.000168009   0.000576633       0.499467   0.000207435
 0.0  -0.458721      0.458196      0.00693677   …   -3.53242   18.6096     

In [ ]:
display(Jt_auto[:,11:13])
display(Jt_num[:,11:13])
display(Jt_num_ip[:,11:13])

In [ ]:
 (J_num[:,1] - x_sol_auglag)/ϵ

In [63]:
maximum(abs.(J_auto-J_num))

0.027098530150311717

In [ ]:
abs.(J_auto-J_num)

In [ ]:
J_num

In [ ]:
J_auto